# Preamble

In [1]:
// Conventional APIs & Instances

trait ConventionalIO{
  def read(): String
  def write(msg: String): Unit
}

object ConsoleConvIO extends ConventionalIO{
  def read() = readLine()
  def write(msg: String) = println(msg)
}

// Functional APIs

trait IO[P[_]]{
  def read(): P[String]
  def write(msg: String): P[Unit]
}

trait Monad[P[_]]{
  def returns[A](a: A): P[A]
  def flatMap[A,B](p: P[A])(f: A => P[B]): P[B]
}

// API Instances

type Id[T] = T 

object ConsoleIO extends IO[Id]{
  def read() = readLine()
  def write(msg: String) = println(msg)
}

case class IOState(read: List[String], written: List[String])

type IOTrans[T] = IOState => (IOState, T)

object StateIO extends IO[IOTrans]{
  def read(): IOTrans[String] = {
    case IOState(msg::readsTail, writes) =>
      (IOState(readsTail, writes), msg)
    case _ => throw new Exception("not enough data to be read")
  }

  def write(msg: String): IOTrans[Unit] = {
    case IOState(reads, writes) =>
      (IOState(reads, msg::writes), ())
  }
}

object StateMonad extends Monad[IOTrans]{
  def flatMap[A,B](p: IOTrans[A])(f: A => IOTrans[B]): IOTrans[B] =
    iostate1 => p(iostate1) match {
      case (iostate2, a) => f(a)(iostate2)
    }

  def returns[A](a: A): IOTrans[A] =
    iostate => (iostate, a)
}

defined trait ConventionalIO
defined object ConsoleConvIO
defined trait IO
defined trait Monad
defined type Id
defined object ConsoleIO
defined class IOState
defined type IOTrans
defined object StateIO
defined object StateMonad

# Exercise 1

Write a program that reads from the standard input and says whether it's
even or ordd.

For instance:

    scala> writeANumber
    << type "8" >>
    8 is even

    scala> writeANumber
    << type "5" >>
    5 is odd

In [2]:
// Helper method
def evenOdd(n: String): String =
if (n.toInt % 2 == 0)
  s"$n es un número par"
else
  s"$n es un número impar"

defined function evenOdd

In [3]:
object Conventional1{
  def writeANumber(io: ConventionalIO): Unit = {
    val num = io.read
    io.write(evenOdd(num))
  }
}

defined object Conventional1

In [ ]:
object Declarative1{
  def writeANumber[F[_]](io: IO[F], m: Monad[F]): F[Unit] =
   ???
}

In [ ]:
val ios = IOState("3" :: Nil, Nil)
val ios2 = IOState("4" :: Nil, Nil)

In [ ]:
assert(Declarative1.writeANumber[IOTrans](StateIO: IO[IOTrans], StateMonad)(ios) == 
    (IOState(Nil, "3 es un número impar" :: Nil), ()))

assert(Declarative1.writeANumber(StateIO, StateMonad)(ios) ==
  (IOState(Nil, "4 es un número par" :: Nil), ()))

# Exercise 2

Same as in `Program1` but now the program asks politely
for a number to the user.

For instance:

    scala> runWriteANumber2
    Please, type a number:
    << type 8 >>
    8 is even


In [ ]:
// Impure version

object Conventional2{
  import Conventional1.writeANumber
 
  def writeANumberBis(io: ConventionalIO): Unit = {
    io.write("Introduce un número por favor:")
    writeANumber(io)
  }
}

In [ ]:
// Declarative version
object Declarative2{
  import Declarative1.writeANumber

  def writeANumberBis[F[_]](io: IO[F], m: Monad[F]): F[Unit] =
    ???
}

In [ ]:
assert(Declarative2.writeANumberBis(StateIO, StateMonad)(ios) ==
  (IOState(Nil, "3 es un número impar" :: "Introduce un número por favor:" :: Nil), ()))

assert(Declarative2.writeANumberBis(StateIO, StateMonad)(ios) ==
  (IOState(Nil, "4 es un número par" :: "Introduce un número por favor:" :: Nil), ()))

# Exercise 3

Write a program that reads lines continously until it's read
the word "exit"

For instance:

    scala> readUntilExit
    <<type "hi" and enter>>
    <<type "bye" and enter>>
    <<type "exit" and enter>>

In [ ]:
object Conventional3{

  def readUntilExit(io: ConventionalIO): Unit = {
    val msg = io.read()
    if (msg == "exit") ()
    else readUntilExit(io)
  }
}

In [ ]:
object Declarative3{

  def readUntilExit[F[_]](io: IO[F], m: Monad[F]): F[Unit] =
    ???
}

In [ ]:
val ios3 = IOState("uno" :: "dos" :: "tres" :: "exit" :: Nil, Nil)
val ios4 = IOState("uno" :: "dos" :: "tres" :: "exit" :: "otro" :: Nil, Nil)

In [ ]:
assert(Declarative3.readUntilExit(StateIO, StateMonad)(ios) ==
        (IOState(Nil, Nil), ()))

assert(Declarative3.readUntilExit(StateIO, StateMonad)(ios) ==
        (IOState("otro" :: Nil, Nil), ()))